In [1]:
import pandas as pd
from pprint import pprint
from os import listdir

In [2]:
# Read in files
dfs = {}
for f in listdir('raw'): 
    if f.endswith('.csv'):
        df = pd.read_csv('raw/' + f)
        name = f.split('.')[0].split('-')[-1]
        print(name)
        dfs[name] = df

combo
mono
ppi
effectcategories
all
targets


In [3]:
dfs['combo']

,STITCH 1,STITCH 2,Polypharmacy Side Effect,Side Effect Name
0,CID000002173,CID000003345,C0151714,hypermagnesemia
1,CID000002173,CID000003345,C0035344,retinopathy of prematurity
2,CID000002173,CID000003345,C0004144,atelectasis
3,CID000002173,CID000003345,C0002063,alkalosis
4,CID000002173,CID000003345,C0004604,Back Ache
...,...,...,...,...
4649436,CID000003461,CID000003954,C0149871,deep vein thromboses
4649437,CID000003461,CID000003954,C0035410,rhabdomyolysis
4649438,CID000003461,CID000003954,C0043096,loss of weight
4649439,CID000003461,CID000003954,C0003962,ascites


In [4]:
dfs['mono']

,STITCH,Individual Side Effect,Side Effect Name
0,CID003062316,C1096328,central nervous system mass
1,CID003062316,C0162830,Photosensitivity reaction
2,CID003062316,C1611725,leukaemic infiltration brain
3,CID003062316,C0541767,platelet adhesiveness abnormal
4,CID003062316,C0242973,Ventricular dysfunction
...,...,...,...
174972,CID005282226,C1096278,anterior chamber inflammation
174973,CID005282226,C0272412,splenic abscess
174974,CID005282226,C0860013,hepatitis b antibody positive
174975,CID005282226,C0853225,international normalised ratio increased


In [5]:

dfs['ppi']

,Gene 1,Gene 2
0,114787,375519
1,114787,285613
2,114787,7448
3,114787,4914
4,114787,51343
...,...,...
715607,5634,5636
715608,5635,5636
715609,5635,5631
715610,5636,5631


In [6]:
dfs['effectcategories']

,Side Effect,Side Effect Name,Disease Class
0,C0017152,gastric inflammation,gastrointestinal system disease
1,C0027858,neuroma,benign neoplasm
2,C0041466,Typhoid,bacterial infectious disease
3,C0032807,Post thrombotic syndrome,cardiovascular system disease
4,C0033860,psoriasis,integumentary system disease
...,...,...,...
556,C0001816,agnosia,developmental disorder of mental health
557,C0302592,carcinoma of the cervix,cancer
558,C0270629,epidural abscess,nervous system disease
559,C0038220,status epilepticus,nervous system disease


In [7]:
dfs['targets']

,STITCH,Gene
0,CID000003488,1559
1,CID000003488,8647
2,CID000077992,3351
3,CID000077992,3350
4,CID000077992,3352
...,...,...
18685,CID000005152,8484
18686,CID000005152,81491
18687,CID000005152,83551
18688,CID000005152,680


# Node counts

In [8]:
# Drug nodes
drug_IDs = set()
for df in dfs.values():
    for col in df.columns:
        if 'STITCH' in col:
            for ID in df[col].unique():
                drug_IDs.add(ID)

assert all([ID.startswith('CID') for ID in drug_IDs])
print(f'{len(drug_IDs)} unique drug IDs found')

2135 unique drug IDs found


In [9]:
# Protein nodes
prot_IDs = set()
for df in dfs.values():
    for col in df.columns:
        if 'Gene' in col:
            for ID in df[col].unique():
                prot_IDs.add(ID)

assert [int(ID) for ID in prot_IDs] # Test they can be converted to int
print(f'{len(prot_IDs)} unique protein IDs found')     


19122 unique protein IDs found


In [10]:
# Side effect nodes
side_IDs = set()
for df in dfs.values():
    if 'Side Effect Name' in df.columns:
        df = df.drop(columns=['Side Effect Name'])
    for col in df.columns:
        if 'Side Effect' in col:
            for ID in df[col].unique():
                side_IDs.add(ID)

assert all([ID.startswith('C') for ID in side_IDs])
print(f'{len(side_IDs)} unique side effect IDs found')  

11501 unique side effect IDs found


# Targets vs Targets-all CSV file overlap

In [11]:
# Check for overlap by merging dfs
union_merged = dfs['all'].merge(dfs['targets'], how='outer', on=['STITCH', 'Gene'])

if len(union_merged) > len(dfs['all']):
    print('\'targets\' contains tuples not contained in \'targets-all\'')
else:
    print('\'targets\' only contains tuples that are contained in \'targets-all\'')

'targets' only contains tuples that are contained in 'targets-all'


In [12]:
# Double checking - cell should run through with no output. Takes a while so only run if necessary.
run = False
if run:
    existing = [[row.STITCH, row.Gene] for i, row in dfs['all'].iterrows()]
    for i, row in dfs['targets'].iterrows():
        to_check = [row.STITCH, row.Gene]
        if to_check not in existing:
            print(to_check)
            break